In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing, ensemble
from numpy import nan
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from com_util import *
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # ingore warnings
%matplotlib inline
color = sns.color_palette()

In [ ]:
cols = ['brent', 'eurrub', 'usdrub', 'micex_cbi_tr', 'micex_rgbi_tr','micex',  'brent', 'rts', 'oil_urals',
        'balance_trade', 'ppi', 'cpi', 'gdp_quart', 'net_capital_export', 'micex_cbi_tr', 'deposits_rate',
       'gdp_quart_growth', 'mortgage_rate', 'average_provision_of_build_contract_moscow']

In [ ]:
df_train = pd.read_csv("../data/train.csv", parse_dates=['timestamp'])
df_test = pd.read_csv("../data/test.csv", parse_dates=['timestamp'])
df_macro = pd.read_csv("../data/macro.csv", parse_dates=['timestamp'], usecols=['timestamp']+cols)
df_train.head()

In [ ]:
# # fix bad address
# fx = pd.read_excel('../data/BAD_ADDRESS_FIX.xlsx').drop_duplicates('id').set_index('id')
# df_train.update(fx, overwrite=True)
# df_test.update(fx, overwrite=True)

In [ ]:
df_train_loc = pd.read_csv("../data/train_lat_lon.csv")
df_test_loc = pd.read_csv("../data/test_lat_lon.csv")
df_train = df_train.merge(df_train_loc, on='id')
df_train.drop(['key', 'tolerance_m'], axis=1, inplace=True)
df_test = df_test.merge(df_test_loc, on='id')
df_test.drop(['key', 'tolerance_m'], axis=1, inplace=True)

In [ ]:
# # undersampling by magic numbers
# trainsub = df_train[df_train.timestamp < '2015-01-01']
# trainsub = trainsub[trainsub.product_type=="Investment"]

# ind_1m = trainsub[trainsub.price_doc <= 1000000].index
# ind_2m = trainsub[trainsub.price_doc == 2000000].index
# ind_3m = trainsub[trainsub.price_doc == 3000000].index

# train_index = set(df_train.index.copy())

# for ind, gap in zip([ind_1m, ind_2m, ind_3m], [10, 3, 2]):
#     ind_set = set(ind)
#     ind_set_cut = ind.difference(set(ind[::gap]))

#     train_index = train_index.difference(ind_set_cut)

# df_train = df_train.loc[train_index]

In [ ]:
y_train = df_train['price_doc'].values
id_test = df_test['id']

df_train.drop(['id', 'price_doc'], axis=1, inplace=True)
df_test.drop(['id'], axis=1, inplace=True)

# Build df_all = (df_train+df_test).join(df_macro)
num_train = len(df_train)
df_all = pd.concat([df_train, df_test])
df_all = df_all.merge(df_macro, on='timestamp')

# Add month-year count
month_year = (df_all.timestamp.dt.month + df_all.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df_all['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (df_all.timestamp.dt.weekofyear + df_all.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df_all['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
df_all['month'] = df_all.timestamp.dt.month
df_all['dow'] = df_all.timestamp.dt.dayofweek

print(df_all.shape)

In [ ]:
# import missingno as msno
# missingValueColumns = df_macro.columns[df_macro.isnull().any()].tolist()
# msno.bar(df_macro[missingValueColumns],\
#             figsize=(20,8),color=(0.5, 0.5, 1),fontsize=12,labels=True,)

In [ ]:
# grouped_df = df_train.groupby('timestamp')['price_doc'].aggregate(np.median).reset_index()

In [ ]:
# plt.figure(figsize=(12,8))
# sns.barplot(grouped_df.timestamp.values, grouped_df.price_doc.values, alpha=0.8, color=color[2])
# plt.ylabel('Median Price', fontsize=12)
# plt.xlabel('Year Month', fontsize=12)
# plt.xticks(rotation='vertical')
# plt.show()

### Data Cleaning

In [ ]:
# doing some cleaning of square
df_all['full_sq'].ix[df_all.full_sq > 1000] = nan
df_all['life_sq'].ix[df_all.life_sq > 1000] = nan

df_all['life_sq'].ix[df_all.full_sq < df_all.life_sq] = nan
df_all['life_sq'].ix[df_all.life_sq < 5] = nan
df_all['full_sq'].ix[df_all.full_sq < 5] = nan
df_all['kitch_sq'].ix[df_all.kitch_sq > df_all.life_sq] = nan

In [ ]:
# cleaning build_year
df_all['build_year'].ix[df_all.build_year == 20052009] = 2007
df_all['build_year'].ix[df_all.build_year < 1500] = nan
df_all['build_year'].ix[df_all.build_year > 2500] = nan

# cleaning state
df_all['state'].ix[df_all.state == 33] = 3

# cleaning floor
df_all['max_floor'].ix[df_all.max_floor == 0] = nan
df_all['floor'].ix[df_all.floor == 0] = nan

### some FE ideas from discussion && kernel

In [ ]:
# separate full_sq
df_all['full_sq_separate'] = df_all['full_sq'].copy()

for i in range(30):
    df_all['full_sq_separate'].ix[(df_all.full_sq > (i+1)*5) & (df_all.full_sq <= ((i+1)*5+5))] = \
    len(df_all['full_sq'].ix[(df_all.full_sq > (i+1)*5) & (df_all.full_sq <= ((i+1)*5+5))])
    
df_all['full_sq_separate'].ix[df_all.full_sq <= 5] = len(df_all['full_sq'].ix[df_all.full_sq <= 5])
df_all['full_sq_separate'].ix[df_all.full_sq <= 5] = len(df_all['full_sq'].ix[df_all.full_sq > 155])

In [ ]:
df_all['life_sq_separate'] = df_all['life_sq'].copy()

for i in range(30):
    df_all['life_sq_separate'].ix[(df_all.life_sq > (i+1)*5) & (df_all.life_sq <= ((i+1)*5+5))] = \
    len(df_all['life_sq'].ix[(df_all.life_sq > (i+1)*5) & (df_all.life_sq <= ((i+1)*5+5))])
    
df_all['life_sq_separate'].ix[df_all.life_sq <= 5] = len(df_all['life_sq'].ix[df_all.life_sq <= 5])
df_all['life_sq_separate'].ix[df_all.life_sq <= 5] = len(df_all['life_sq'].ix[df_all.life_sq > 155])

In [ ]:
# Add null value counts
df_all['null_count'] = df_all.isnull().sum(axis=1)

In [ ]:
# Remove timestamp column (may overfit the model in train)
df_all.drop(['timestamp'], axis=1, inplace=True)

In [ ]:
#每个经纬度分类一下（1平方千米一个类）
df_all["jwd_class"]=map(lambda x,y:(int(x*100)%100)*100+(int(-y*100)%100),df_all["lat"].fillna(0),df_all["lon"].fillna(0))

In [ ]:
# GroupBy 经纬度
df_all = merge_median(df_all, ["jwd_class"], "full_sq", "fullsq_median_jwd")
df_all = merge_median(df_all, ["jwd_class"], "life_sq", "lifesq_median_jwd")
df_all = merge_median(df_all, ["jwd_class"], "floor", "floor_median_jwd")

In [ ]:
# plt.figure(figsize=(12,8))
# sns.countplot(x="usdrub", data=df_macro)
# plt.ylabel('Count', fontsize=12)
# plt.xlabel('Variable', fontsize=12)
# plt.xticks(rotation='vertical')
# plt.show()

In [ ]:
# plt.figure(figsize=(12,8))
# sns.countplot(x="full_sq_separate", data=df_all)
# plt.ylabel('Count', fontsize=12)
# plt.xlabel('Variable', fontsize=12)
# plt.xticks(rotation='vertical')
# plt.show()

## Correlation Analysis

In [ ]:
featureImportance = model.get_fscore()
features = pd.DataFrame()
features['features'] = featureImportance.keys()
features['importance'] = featureImportance.values()
features.sort_values(by=['importance'],ascending=False,inplace=True)
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
plt.xticks(rotation=60)
sn.barplot(data=features.head(30),x="features",y="importance",ax=ax,orient="v")

In [ ]:
topFeatures = features["features"].tolist()[:15]
topFeatures.append("price_doc")
corrMatt = train[topFeatures].corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sn.heatmap(corrMatt, mask=mask,vmax=.8, square=True,annot=True)

In [ ]:
# Deal with categorical values
df_numeric = df_all.select_dtypes(exclude=['object'])
df_obj = df_all.select_dtypes(include=['object']).copy()

for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]

df_values = pd.concat([df_numeric, df_obj], axis=1)

In [ ]:
# Dealing with missed values
# from sklearn.base import TransformerMixin
# class DataFrameImputer(TransformerMixin):
#     def fit(self, X, y=None):
#         self.fill = pd.Series([X[c].value_counts().index[0]
#         if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
#         index=X.columns)
#         return self
#     def transform(self, X, y=None):
#         return X.fillna(self.fill)
# df_values = DataFrameImputer().fit_transform(df_values)

In [ ]:
# Convert to numpy values
X_all = df_values.values
print(X_all.shape)

X_train = X_all[:num_train]
X_test = X_all[num_train:]

df_columns = df_values.columns

In [ ]:
train_y = np.log1p(y_train)

In [ ]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1,
}

dtrain = xgb.DMatrix(X_train, y_train, feature_names=df_columns)
dtest = xgb.DMatrix(X_test, feature_names=df_columns)

In [ ]:
cv_output = xgb.cv(xgb_params, dtrain, nfold=5, num_boost_round=1000, early_stopping_rounds=20,
    verbose_eval=20, show_stdv=False)

In [ ]:
# cv_scores = []
# kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
# for dev_index, val_index in kf.split(range(X_train.shape[0])):
#         dev_X, val_X = X_train[dev_index,:], X_train[val_index,:]
#         dev_y, val_y = train_y[dev_index], train_y[val_index]
#         reg.fit(dev_X, dev_y)
#         preds = reg.predict(val_X)
#         cv_scores.append(np.sqrt(mean_squared_error(val_y, preds)))
#         print cv_scores

In [ ]:
print len(cv_output)

In [ ]:
num_boost_rounds = 500
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= num_boost_rounds)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 16))
xgb.plot_importance(model, max_num_features=50, height=0.5, ax=ax)
plt.show()

In [ ]:
y_pred = model.predict(dtest)

df_sub = pd.DataFrame({'id': id_test, 'price_doc': y_pred})

df_sub.to_csv('sub.csv', index=False)

In [ ]:
df_sub['price_doc'].mean()

In [ ]:
sub.to_csv('sub.csv', index=False)